In [ ]:
%matplotlib inline

# 신경망 모델 구성하기

신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있다.
[`torch.nn`](<https://pytorch.org/docs/stable/nn.html>) 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공한다.
PyTorch의 모든 모듈은 [`nn.Module`](<https://pytorch.org/docs/stable/generated/torch.nn.Module.html>)의 하위 클래스(subclass)
이다. 신경망은 그 자체로 다른 모듈 혹은 계층(layer)들로 구성된 하나의 모듈이다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있게 한다.

이제 FashionMNIST 데이터셋의 이미지들을 분류하는 신경망을 구성해보자.


In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

학습을 위한 장치 얻기
------------------------------------------------------------------------------------------

가능하다면 GPU와 같은 하드웨어 가속기에서 모델을 학습하는 것이 효율적이다.
[`torch.cuda`](<https://pytorch.org/docs/stable/notes/cuda.html>)를 사용할 수 있는지
확인하고 그렇지 않으면 CPU를 계속 사용한다.

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


클래스 정의하기
------------------------------------------------------------------------------------------

신경망 모델을 ``nn.Module``의 하위클래스로 정의하고, ``__init__``에서 신경망 계층들을 초기화한다.
``nn.Module``을 상속받은 모든 클래스는 ``forward`` 메소드에 입력 데이터에 대한 연산들을 구현한다.



In [ ]:
img_height = 28     # 이미지의 높이
img_width = 28      # 이미지의 너비
num_channels = 1    # 흑백 이미지이므로 1
num_classes = 10    # 분류할 이미지의 클래스는 10가지

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # 2차원 모델을 1차원 모델로 변환 
        self.linear_relu_stack = nn.Sequential( # 3개의 리니어 모듈로 구성 
            nn.Linear(img_height*img_width*num_channels, 512), # 노드 갯수는 512개 (출력 사이즈도 512가 됨)
            nn.ReLU(),
            nn.Linear(512, 512), # 노드 갯수도 512개 
            nn.ReLU(),
            nn.Linear(512, num_classes), # 마지막 레이어는 Activation fun 없음 
        )

    def forward(self, x): # 입력x : 이미지 
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

거의 유사한 구조의 네트워크를 약간 다르게 정의해보았다.

In [ ]:
class NeuralNetwork2(nn.Module): # 512와 1024 차이 뿐 다른건 같은 코드 
    def __init__(self):
        super(NeuralNetwork2, self).__init__()
        self.fc1 = nn.Linear(img_height*img_width*num_channels, 1024) 
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.view(x.size(0), -1)     # flatten the input image / x.size -> batch size / -1 -> undefined // 그냥 이런것도 있다 
        out = self.relu(self.fc1(x))
        out = self.relu(self.fc2(out))
        out = self.fc3(out)
        return out

``NeuralNetwork``의 인스턴스(instance)를 생성하고 이를 ``device``로 이동한 뒤,
구조(structure)를 출력한다.



In [ ]:
model = NeuralNetwork().to(device) # 필요한 작업... CPU 쓸건지 GPU 쓸건지 
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


모델에 입력 데이터를 전달하면 모델의 ``forward`` 메서드가 자동으로 실행된다. ``model.forward()``를 사용자가 직접 호출하지는 않는다.

입력 데이터에 대해 모델을 실행하면 각 분류(class)에 대한 원시(raw) 예측값이 저장된 10-차원 텐서가 반환된다. 이 값을 각 분류에 속할 확률(각각은 0에서 1사이의 실수이고 합이 1이 되는 실수들)로 변환하고 싶다면 ``nn.Softmax`` 모듈을 적용하면 된다. //확률을 나타내고 싶을 때 사용

In [ ]:
X = torch.rand(2, 1, 28, 28, device=device)  # 2 random grayscale images / 픽셀값은 랜덤인 2개의 이미지
logits = model(X)
print('logits: {}'.format(logits))
pred_probab = nn.Softmax(dim=1)(logits) # softMax가 적용되는 함수가 1차원 이기 때문에 dim=1
print('pred_probab: {}'.format(pred_probab))

y_pred = pred_probab.argmax(1) # argmax? 10개의 텐서 중 몇번째 값이 최댓값인지 찾아주는 함수 '(1)'은 '축'이라는 뜻(?) //아래 Predicted class: tensor([0, 8]) 는 최대값이 각각 이미지 별로 0과 8이다.
print(f"Predicted class: {y_pred}")

logits: tensor([[ 0.0112, -0.0173, -0.0627,  0.0251, -0.0767, -0.0720, -0.1072,  0.0394,
         -0.1336, -0.0520],
        [ 0.0338, -0.0205, -0.0720, -0.0101,  0.0035, -0.0418, -0.1244,  0.0421,
         -0.1517, -0.0211]], device='cuda:0', grad_fn=<AddmmBackward0>)
pred_probab: tensor([[0.1056, 0.1026, 0.0981, 0.1071, 0.0967, 0.0971, 0.0938, 0.1086, 0.0913,
         0.0991],
        [0.1071, 0.1014, 0.0963, 0.1025, 0.1039, 0.0993, 0.0914, 0.1079, 0.0889,
         0.1013]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: tensor([7, 7], device='cuda:0')


#### `nn.Softmax`

신경망의 마지막 선형 계층은 [`nn.Softmax`](<https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>) 모듈에 전달될
[-\infty, \infty] 범위의 원시 값(raw value)인 `logits`를 반환한다. `nn.Softmax` 계층은 logits는 모델의 각 분류(class)에 대한 예측 확률을 나타내도록
[0, 1] 범위로 비례하여 조정(scale)한다. ``dim`` 매개변수는 값의 합이 1이 되는 차원을 나타낸다.



In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
pred_probab

tensor([[0.1056, 0.1026, 0.0981, 0.1071, 0.0967, 0.0971, 0.0938, 0.1086, 0.0913,
         0.0991],
        [0.1071, 0.1014, 0.0963, 0.1025, 0.1039, 0.0993, 0.0914, 0.1079, 0.0889,
         0.1013]], device='cuda:0', grad_fn=<SoftmaxBackward0>)

:**Note:** `SoftMax`함수 $\sigma : R^K \longrightarrow (0, 1)^K$의 정의: $\sigma(z)_i = \frac{e^{z_i}}{\sum_{j=1}^K e^{z_j}}$  .......>> 총합은 1

------------------------------------------------------------------------------------------




모델 계층(Layer)
------------------------------------------------------------------------------------------

FashionMNIST 모델의 계층들을 살펴보자. 이를 설명하기 위해, 28x28 크기의 랜덤 이미지 3개로 구성된
미니배치를 가져와, 신경망을 통과할 때 어떤 일이 발생하는지 알아본다.



In [ ]:
rand_img = torch.rand(3,1,10,10)
print(rand_img)
flatten = nn.Flatten()
flatten_img = flatten(rand_img)
print(flatten_img)

tensor([[[[0.3703, 0.0600, 0.7030, 0.1387, 0.0174, 0.7948, 0.8735, 0.9354,
           0.6006, 0.3434],
          [0.8130, 0.3379, 0.0637, 0.7777, 0.8361, 0.0972, 0.9796, 0.4348,
           0.2590, 0.8838],
          [0.9932, 0.3749, 0.9659, 0.8884, 0.1928, 0.3626, 0.6691, 0.9380,
           0.0327, 0.6007],
          [0.5853, 0.3734, 0.9444, 0.6145, 0.4300, 0.8586, 0.6391, 0.4828,
           0.8082, 0.8679],
          [0.7001, 0.9753, 0.9237, 0.8612, 0.1558, 0.6938, 0.7100, 0.5217,
           0.5387, 0.7099],
          [0.8806, 0.7669, 0.4596, 0.2713, 0.1233, 0.2095, 0.5250, 0.9784,
           0.7892, 0.5210],
          [0.7212, 0.1010, 0.9211, 0.2039, 0.6626, 0.6257, 0.2944, 0.8075,
           0.8869, 0.1149],
          [0.0026, 0.6174, 0.9844, 0.5670, 0.3040, 0.1114, 0.6744, 0.0792,
           0.2275, 0.3920],
          [0.4521, 0.2645, 0.4835, 0.3738, 0.5864, 0.3693, 0.6202, 0.2095,
           0.9094, 0.3329],
          [0.4387, 0.7591, 0.9660, 0.1640, 0.2754, 0.6682, 0.6456, 0.8884

In [ ]:
input_images = torch.rand(3, 1, 28, 28) # 2 random grayscale images
print(input_images)
print(input_images.size())

tensor([[[[0.5899, 0.1377, 0.5635,  ..., 0.7419, 0.5389, 0.1329],
          [0.4346, 0.5824, 0.1957,  ..., 0.9177, 0.9711, 0.8584],
          [0.5505, 0.7385, 0.4338,  ..., 0.3232, 0.7114, 0.1898],
          ...,
          [0.2887, 0.1678, 0.9462,  ..., 0.7314, 0.4096, 0.9896],
          [0.6946, 0.0042, 0.1949,  ..., 0.6815, 0.3720, 0.6154],
          [0.2296, 0.9481, 0.8549,  ..., 0.7209, 0.1182, 0.0214]]],


        [[[0.2177, 0.5515, 0.6387,  ..., 0.9155, 0.4830, 0.5330],
          [0.8900, 0.0887, 0.8392,  ..., 0.7162, 0.9012, 0.3201],
          [0.7249, 0.7423, 0.7093,  ..., 0.1522, 0.4259, 0.6443],
          ...,
          [0.5613, 0.9525, 0.0628,  ..., 0.7070, 0.5746, 0.8042],
          [0.9412, 0.5226, 0.9981,  ..., 0.3875, 0.2468, 0.2501],
          [0.4412, 0.7136, 0.2252,  ..., 0.2590, 0.7530, 0.8188]]],


        [[[0.0178, 0.8296, 0.6243,  ..., 0.2192, 0.6648, 0.8247],
          [0.3952, 0.3111, 0.8062,  ..., 0.1829, 0.2982, 0.5585],
          [0.1321, 0.4822, 0.0016,  ..

#### `nn.Flatten`

[`nn.Flatten`](<https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>) 계층은
각 28x28의 2D 이미지를 784 픽셀 값을 갖는 1차원 배열로 변환한다. dim=0의 미니배치 차원은 유지된다.



In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_images)
print(flat_image.size())

torch.Size([3, 784])


#### `nn.Linear`

[선형 계층](<https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>)은 저장된 가중치(weight)와
편향(bias)을 사용하여 입력에 선형 변환(linear transformation)을 적용하는 모듈이다.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # 노드가 20개인 리니어 레이어 (곧, 출력 벡터의 사이즈즈)
hidden1 = layer1(flat_image)
print(hidden1.size())
print(hidden1)

torch.Size([3, 20])
tensor([[-0.5696, -0.0971,  0.4020,  0.1580,  0.5307,  0.1039, -0.5530, -0.2462,
          0.0940, -0.5691,  0.0738, -0.1913, -0.0880,  0.2391, -0.1928, -0.3406,
          0.7934,  0.2581, -0.1567, -0.0165],
        [-0.2834,  0.2402,  0.3911,  0.0409, -0.0400,  0.2782, -0.4495, -0.3883,
         -0.1695,  0.1305,  0.1453, -0.1059, -0.4036,  0.1702, -0.1996, -0.1264,
          0.4059,  0.2540, -0.2857, -0.0910],
        [-0.3287,  0.0295,  0.2424, -0.0582,  0.0675, -0.1619, -0.6016, -0.2853,
         -0.1620, -0.4376, -0.2458,  0.3466, -0.2085,  0.1128, -0.1083,  0.1271,
          0.2557,  0.0282, -0.0290, -0.3726]], grad_fn=<AddmmBackward0>)


#### `nn.ReLU`

비선형 활성화(activation)는 모델의 입력과 출력 사이에 비선형적인 복잡한 관계(mapping)를 만든다.
비선형 활성화는 선형 변환 후에 적용되어 *비선형성(nonlinearity)* 을 도입하고, 신경망이 다양한 현상을 학습할 수 있도록 돕는다.

이 모델에서는 [`nn.ReLU`](<https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>)를 사용하지만, 다양한 다른 활성화 함수를 사용할 수도 있다. 

**Note:** $ReLu(x) = \max(0, x)$이다.



In [ ]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1) 
print(f"After ReLU: {hidden1}") # 음수는 0으로, 양수는 그대로 출력된 것을 볼 수 있다. 

Before ReLU: tensor([[-0.5696, -0.0971,  0.4020,  0.1580,  0.5307,  0.1039, -0.5530, -0.2462,
          0.0940, -0.5691,  0.0738, -0.1913, -0.0880,  0.2391, -0.1928, -0.3406,
          0.7934,  0.2581, -0.1567, -0.0165],
        [-0.2834,  0.2402,  0.3911,  0.0409, -0.0400,  0.2782, -0.4495, -0.3883,
         -0.1695,  0.1305,  0.1453, -0.1059, -0.4036,  0.1702, -0.1996, -0.1264,
          0.4059,  0.2540, -0.2857, -0.0910],
        [-0.3287,  0.0295,  0.2424, -0.0582,  0.0675, -0.1619, -0.6016, -0.2853,
         -0.1620, -0.4376, -0.2458,  0.3466, -0.2085,  0.1128, -0.1083,  0.1271,
          0.2557,  0.0282, -0.0290, -0.3726]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.4020, 0.1580, 0.5307, 0.1039, 0.0000, 0.0000, 0.0940,
         0.0000, 0.0738, 0.0000, 0.0000, 0.2391, 0.0000, 0.0000, 0.7934, 0.2581,
         0.0000, 0.0000],
        [0.0000, 0.2402, 0.3911, 0.0409, 0.0000, 0.2782, 0.0000, 0.0000, 0.0000,
         0.1305, 0.1453, 0.0000, 0.0000, 0.1702, 0.00

#### `nn.Sequential`

[`nn.Sequential`](<https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>)은 순서를 갖는
모듈의 컨테이너이다. 데이터는 모듈들을 나열된 순서대로 통과한다. 순차 컨테이너(sequential container)를 사용하여 아래의 ``seq_modules``와 같은 신경망을 빠르게 만들 수 있다.



In [ ]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_images = torch.rand(3, 1, 28, 28)
logits = seq_modules(input_images)
logits

tensor([[-0.0333,  0.2017,  0.2365, -0.1200,  0.0542,  0.2103,  0.0135, -0.3470,
         -0.2061,  0.0491],
        [ 0.0039,  0.1499,  0.2371, -0.1485,  0.0523,  0.3345,  0.0496, -0.3788,
         -0.2201,  0.0369],
        [ 0.0689,  0.1479,  0.2206, -0.1252,  0.0388,  0.2189, -0.0534, -0.2929,
         -0.1407,  0.0749]], grad_fn=<AddmmBackward0>)

모델 매개변수
------------------------------------------------------------------------------------------

신경망에서 많은 계층들은 매개변수화(parameterize)되어 있다. 즉, 학습 중에 최적화되는 가중치(weight)와 편향(bias)을 가진다. ``nn.Module``을 상속하면 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, 모델의 ``parameters()`` 및
``named_parameters()`` 메소드로 모든 매개변수에 접근할 수 있게 된다.

이 예제에서는 각 매개변수들을 순회하며(iterate), 매개변수의 크기와 값을 출력한다.




In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters(): # 두 개의 값을 엑세스 가능하다 
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n") # 두 개만 프린트 해보자자

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0296, -0.0281,  0.0208,  ...,  0.0304, -0.0031,  0.0042],
        [ 0.0146, -0.0197,  0.0139,  ..., -0.0031,  0.0133, -0.0178]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0239, 0.0231], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0084, -0.0399,  0.0067,  ...,  0.0089,  0.0374, -0.0070],
        [ 0.0076,  0.0242,  0.0286,  ...,  0.0343, -0.0438, -0.0305]],
       device='cuda:0', grad_fn=<Slic

------------------------------------------------------------------------------------------




더 읽어보기
------------------------------------------------------------------------------------------
- [`torch.nn API`](<https://pytorch.org/docs/stable/nn.html>)

